In [1]:
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt

data = np.array([
    -4.127935, -2.152332, 3.046302, -10.77442, -6.985155, 5.536106, 1.294746,
    7.638998, -5.650118, -4.478067, -0.4551435, -3.969413, -0.4225518,
    -2.462094, 1.886675, 3.095567, 4.310003, 3.305034, 0.8548273, 4.368213])

# Plot of the background normal distribution and the data. This shows what we
# know about the model. What we don't know is the other normal distribution.

plot_mu_range = [-12, 12]
plot_sigma_range = [0, 4]
plt.figure(figsize=(8, 4))
plot_x = np.linspace(plot_mu_range[0], plot_mu_range[1], 500)
plot_y = sts.norm.pdf(plot_x, loc=0, scale=4)
plt.plot(plot_x, plot_y, 'k-')
plt.plot(data, [-0.005] * len(data), 'ko', alpha=0.5)
plt.xlim(-12, 12)
plt.xlabel('x')
plt.ylabel('probability density')
plt.xticks([-10, -5, 0, 5, 10])
plt.title('Background distribution and data')
plt.show()

In [2]:
def unnormalized_posterior(mu, sigma, data):
    '''
    The unnormalized posterior pdf over μ and σ. The pdf depends on the
    parameters, `mu` and `sigma`, as well as the `data`.
    '''
    data = data.reshape((-1,1))  # Make the data a column vector
    return (
        sts.gamma.pdf(sigma, a=2, scale=1) *
        sts.norm.pdf(mu, loc=0, scale=5) *
        np.prod(
            0.7 * sts.norm.pdf(data, loc=0, scale=4) +
            0.3 * sts.norm.pdf(data, loc=mu, scale=np.where(sigma <= 0, 1e-6, sigma)),
            axis=0))

# Plot of the unormalized posterior using a 2-d contour plot.
grid_size = 300
mu = np.linspace(plot_mu_range[0], plot_mu_range[1], grid_size)
sigma = np.linspace(plot_sigma_range[0], plot_sigma_range[1], grid_size)
mu_grid, sigma_grid = np.meshgrid(mu, sigma)
posterior = unnormalized_posterior(
    mu_grid.flat[:], sigma_grid.flat[:], data).reshape((grid_size, grid_size))

plt.figure(figsize=(10, 9))
filled = plt.contourf(mu, sigma, posterior, 11, cmap=plt.cm.YlOrRd_r)
lines = plt.contour(mu, sigma, posterior, 11, colors='black')
plt.title('Unnormalized posterior over μ and σ.')
plt.xlabel('μ')
plt.ylabel('σ')
color_bar = plt.colorbar(filled)
color_bar.add_lines(lines)
plt.show()

## Task 1

* There isn't just one right answer here. Any proposal distributions that covers the posterior pdf should be fine.
* Note that we do want some $\sigma > 4$ samples since the posterior pdf still has mass there.

In [3]:
def good_proposal_rvs(size):
    return {
        'mu': sts.norm.rvs(loc=0, scale=5, size=size),
        'sigma': sts.norm.rvs(loc=3, scale=2, size=size)}


# Generate samples
num_samples = 10000

# The bad proposal distribution is a product of independent normal distributions
# over mu and sigma with means [-5, 1.5] and standard deviations [2, 0.5].
samples = good_proposal_rvs(num_samples)

# Plot of the first 1000 samples on the level plot
plt.figure(figsize=(10, 9))
filled = plt.contourf(mu, sigma, posterior, 11, cmap=plt.cm.YlOrRd_r)
lines = plt.contour(mu, sigma, posterior, 11, colors='black')
plt.plot(samples['mu'][:1000], samples['sigma'][:1000], 'o', color='#00c0ff', alpha=0.5)
plt.title('Unnormalized log-posterior with samples from the proposal distribution.')
plt.xlabel('μ')
plt.ylabel('σ')
color_bar = plt.colorbar(filled)
color_bar.add_lines(lines)
plt.show()

## Task 2
Once you have your samples, use them to approximate the following—

* The mean and standard deviation of $\mu$.
* The mean and standard deviation of $\sigma$.

Note that you can use the same samples to calculate all four of these values. You do not have to generate new samples every time.

You will have to enter the pdf of your proposal distribution below to calculate the importance weights.

In [4]:
def good_proposal_pdf(mu, sigma):
    return sts.norm.pdf(mu, loc=0, scale=5) * sts.norm.pdf(sigma, loc=3, scale=2)

# Calculate the importance weights, p(sample) / q(sample). You need the
# importance weights to compute expected values under the target distribution.
p_values = unnormalized_posterior(samples['mu'], samples['sigma'], data)
q_values = good_proposal_pdf(samples['mu'], samples['sigma'])
importance_weights = p_values / q_values

In [5]:
# TASK: Approximate the mean and standard deviation of mu and sigma using
# mu_samples, sigma_samples, and the importance weights calculated above.

total_importance = np.sum(importance_weights)

expected_mu = np.sum(samples['mu'] * importance_weights) / total_importance
expected_mu_squared = np.sum(samples['mu'] ** 2 * importance_weights) / total_importance

expected_sigma = np.sum(samples['sigma'] * importance_weights) / total_importance
expected_sigma_squared = np.sum(samples['sigma'] ** 2 * importance_weights) / total_importance

mu_mean = expected_mu
mu_var = expected_mu_squared - expected_mu ** 2

sigma_mean = expected_sigma
sigma_var = expected_sigma_squared - expected_sigma ** 2

print('μ mean: ', mu_mean)
print('μ variance: ', mu_var)
print('σ mean: ', sigma_mean)
print('σ variance: ', sigma_var)

μ mean:  -0.32694112484283167
μ variance:  13.63710720482387
σ mean:  2.888110433005975
σ variance:  2.2925511026553274
